<a href="https://colab.research.google.com/github/ChowchowWorks/Customer_service_rag/blob/main/Rag_Pipeline_Prototype_Version_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 1: Setting up the Environment

In [ ]:
import os

!pip install langchain_community langchain chromadb transformers sentence-transformers
!pip install -U langchain-huggingface

!pip install pypdf
!pip install langchain langchain-community langchainhub tiktoken
!pip install -U sentence-transformers


os.environ['LANGCHAIN_API_KEY'] = "API_TOKEN"
os.environ['HUGGINGFACEHUB_API_TOKEN'] = "API_TOKEN"
os.environ['USER_AGENT'] = 'MyColabApp/1.0 (Python/3.9; GoogleColab)'

# Section 2: Loading Documents

(a) Loading documents into a global variable

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

# For the purpose of testing, this version uses a pdf loader
loader = PyPDFDirectoryLoader("/content/RAG_tester")
documents = loader.load()

(b) Text Splitting

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 100) # These are hyperparameters, can attempt tuning this using bayesian optimisation in the future
texts = splitter.split_documents(documents)

# Section 3: Embedding

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Section 4: Generator

(a) Client and Class

In [ ]:
from huggingface_hub import InferenceClient
client = InferenceClient(model="mistralai/Mistral-7B-Instruct-v0.3", token = os.environ['HUGGINGFACEHUB_API_TOKEN'])

from langchain_core.runnables import Runnable

class HuggingFaceChatRunnable(Runnable):
    def __init__(self, client, prompt_template, temperature, max_tokens):
        self.client = client
        self.prompt_template = prompt_template
        self.temperature = temperature
        self.max_tokens = max_tokens

    def invoke(self, inputs: dict, config: dict = None) -> str:
        prompt_str = self.prompt_template.format(**inputs)

        response = self.client.chat_completion(
            messages=[
                {"role": "user", "content": prompt_str}
            ],
            temperature=self.temperature,
            max_tokens=self.max_tokens
        )
        return response.choices[0].message["content"]

# Section 5: RAPTOR (Gaussian Mixture)

(a) Clustering Structure

In [ ]:
from typing import Dict, List, Optional, Tuple
import numpy as np
import pandas as pd
import umap
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from sklearn.mixture import GaussianMixture

seed = 1000

# -- Credit for code below belongs to parthsarthi03, llama_index and lang_chain -- #

def global_cluster_embeddings(
    embeddings: np.ndarray,
    dim: int,
    n_neighbors: Optional[int] = None,
    metric: str = "cosine",
) -> np.ndarray:
    """
    Perform global dimensionality reduction on the embeddings using UMAP.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - dim: The target dimensionality for the reduced space.
    - n_neighbors: Optional; the number of neighbors to consider for each point.
                   If not provided, it defaults to the square root of the number of embeddings.
    - metric: The distance metric to use for UMAP.

    Returns:
    - A numpy array of the embeddings reduced to the specified dimensionality.
    """
    if n_neighbors is None:
        n_neighbors = int((len(embeddings) - 1) ** 0.5)
    return umap.UMAP(
        n_neighbors=n_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)

def local_cluster_embeddings(
    embeddings: np.ndarray, dim: int, num_neighbors: int = 10, metric: str = "cosine"
) -> np.ndarray:
    """
    Perform local dimensionality reduction on the embeddings using UMAP, typically after global clustering.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - dim: The target dimensionality for the reduced space.
    - num_neighbors: The number of neighbors to consider for each point.
    - metric: The distance metric to use for UMAP.

    Returns:
    - A numpy array of the embeddings reduced to the specified dimensionality.
    """
    return umap.UMAP(
        n_neighbors=num_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)

def get_optimal_clusters(
    embeddings: np.ndarray, max_clusters: int = 50, random_state: int = seed
) -> int:
    """
    Determine the optimal number of clusters using the Bayesian Information Criterion (BIC) with a Gaussian Mixture Model.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - max_clusters: The maximum number of clusters to consider.
    - random_state: Seed for reproducibility.

    Returns:
    - An integer representing the optimal number of clusters found.
    """
    max_clusters = min(max_clusters, len(embeddings))
    n_clusters = np.arange(1, max_clusters)
    bics = []
    for n in n_clusters:
        gm = GaussianMixture(n_components=n, random_state=random_state)
        gm.fit(embeddings)
        bics.append(gm.bic(embeddings))
    return n_clusters[np.argmin(bics)]

def GMM_cluster(embeddings: np.ndarray, threshold: float, random_state: int = 0):
    """
    Cluster embeddings using a Gaussian Mixture Model (GMM) based on a probability threshold.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - threshold: The probability threshold for assigning an embedding to a cluster.
    - random_state: Seed for reproducibility.

    Returns:
    - A tuple containing the cluster labels and the number of clusters determined.
    """
    n_clusters = get_optimal_clusters(embeddings)
    gm = GaussianMixture(n_components=n_clusters, random_state=random_state)
    gm.fit(embeddings)
    probs = gm.predict_proba(embeddings)
    labels = [np.where(prob > threshold)[0] for prob in probs]
    return labels, n_clusters


def perform_clustering(
    embeddings: np.ndarray,
    dim: int,
    threshold: float,
) -> List[np.ndarray]:
    """
    Perform clustering on the embeddings by first reducing their dimensionality globally, then clustering
    using a Gaussian Mixture Model, and finally performing local clustering within each global cluster.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - dim: The target dimensionality for UMAP reduction.
    - threshold: The probability threshold for assigning an embedding to a cluster in GMM.

    Returns:
    - A list of numpy arrays, where each array contains the cluster IDs for each embedding.
    """
    if len(embeddings) <= dim + 1:
        # Avoid clustering when there's insufficient data
        return [np.array([0]) for _ in range(len(embeddings))]

    # Global dimensionality reduction
    reduced_embeddings_global = global_cluster_embeddings(embeddings, dim)
    # Global clustering
    global_clusters, n_global_clusters = GMM_cluster(
        reduced_embeddings_global, threshold
    )

    all_local_clusters = [np.array([]) for _ in range(len(embeddings))]
    total_clusters = 0

    # Iterate through each global cluster to perform local clustering
    for i in range(n_global_clusters):
        # Extract embeddings belonging to the current global cluster
        global_cluster_embeddings_ = embeddings[
            np.array([i in gc for gc in global_clusters])
        ]

        if len(global_cluster_embeddings_) == 0:
            continue
        if len(global_cluster_embeddings_) <= dim + 1:
            # Handle small clusters with direct assignment
            local_clusters = [np.array([0]) for _ in global_cluster_embeddings_]
            n_local_clusters = 1
        else:
            # Local dimensionality reduction and clustering
            reduced_embeddings_local = local_cluster_embeddings(
                global_cluster_embeddings_, dim
            )
            local_clusters, n_local_clusters = GMM_cluster(
                reduced_embeddings_local, threshold
            )

        # Assign local cluster IDs, adjusting for total clusters already processed
        for j in range(n_local_clusters):
            local_cluster_embeddings_ = global_cluster_embeddings_[
                np.array([j in lc for lc in local_clusters])
            ]
            indices = np.where(
                (embeddings == local_cluster_embeddings_[:, None]).all(-1)
            )[1]
            for idx in indices:
                all_local_clusters[idx] = np.append(
                    all_local_clusters[idx], j + total_clusters
                )

        total_clusters += n_local_clusters

    return all_local_clusters

(b) Helper functions for recursive clustering

In [ ]:
from langchain.prompts import ChatPromptTemplate
# -- codes below are sourced and inspired by langchain -- #

def embedding_to_np(texts): # convert text embeddings into a numpy array
  text_embeddings = embeddings.embed_documents([t.page_content for t in texts])
  return np.array(text_embeddings)

def embed_cluster_texts(texts): # perform clustering on the numpy array
  np_embeddings = embedding_to_np(texts)
  cluster_labels = perform_clustering(np_embeddings, 10, 0.1)
  df = pd.DataFrame()
  df["text"] = texts
  df["embd"] = list(np_embeddings)
  df["cluster"] = cluster_labels
  return df

def format_texts(df): # reformat the embedded text
  unique = df["text"].tolist()
  # Extract page_content from Document objects before joining
  unique_str = [doc.page_content for doc in unique]
  return "--- --- \n --- --- ".join(unique_str)

def embed_cluster_summarised_text(texts, level) -> Tuple[pd.DataFrame, pd.DataFrame]: # embeds, clusters and summarises a list of texts
  df_clusters = embed_cluster_texts(texts)
  expanded_list = []
  for index, row in df_clusters.iterrows(): # expand dataframe entries to document-cluster pairings for straightforward processing
        for cluster in row["cluster"]:
            expanded_list.append(
                {"text": row["text"], "embd": row["embd"], "cluster": cluster}
            )
  expanded_df = pd.DataFrame(expanded_list)
  all_clusters = expanded_df["cluster"].unique()
  print(f"--Generated {len(all_clusters)} clusters--")
  # Summary
  template = """Here is a subset of books focussed on behavioral economics.

    Behavioral Economics is a branch of economics that incorporates insights from psychology and emphasizes the importance of supposed irrelevant factors in economic analysis.

    Give a detailed summary of the documentation provided.

    Documentation:
    {context}
    """

  prompt = ChatPromptTemplate.from_template(template)
  summariser = HuggingFaceChatRunnable(client, prompt, 0.0, 1024)

  summaries = [] # Format text within each cluster for summarization
  for i in all_clusters:
        df_cluster = expanded_df[expanded_df["cluster"] == i]
        formatted_txt = format_texts(df_cluster)
        summaries.append(summariser.invoke({"context": formatted_txt}))
  df_summary = pd.DataFrame(
        {
            "summaries": summaries,
            "level": [level] * len(summaries),
            "cluster": list(all_clusters),
        }
    )
  return df_clusters, df_summary

def recursive_embed_cluster_summarize(
    texts: List[str], level: int = 1, n_levels: int = 3
) -> Dict[int, Tuple[pd.DataFrame, pd.DataFrame]]:
  results = {} # dictionary to store results
  df_clusters, df_summary = embed_cluster_summarised_text(texts, level)
  results[level] = (df_clusters, df_summary)

  # determine if further recursion and meaningful
  unique_clusters = df_clusters["cluster"].nunique()
  if level < n_levels and unique_clusters > 1:
    new_texts = df_summary["summaries"].tolist() # Use summaries as the input texts for the next level of recursion
    next_level_results = recursive_embed_cluster_summarize(
        new_texts, level + 1, n_levels
    )
    results.update(next_level_results)

    return results


(c) Performing recursive cluster

In [ ]:
leaf_texts = texts
results = recursive_embed_cluster_summarize(leaf_texts, level = 1, n_levels= 3)

(d) Set up the retriever

In [ ]:
vectorstore = Chroma.from_documents(results, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": k})

# Section 6: Routing Prompts

(a) Produce some fewshot examples to incorporate into routing prompt

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import FewShotChatMessagePromptTemplate

# Some fewshot examples
examples =[
    {
        "input": "What is creatine?",
        "output": "DEFINE",
    },
    {
        "input": "Why do athletes take protein after workouts?",
        "output": "EXPLAIN",
    },
    {
        "input": "How do I calculate my calorie needs?",
        "output": "PROCEDURE",
    },
    {
        "input": "Should I take whey or casein protein?",
        "output": "COMPARISON",
    },
    {
        "input": "What is the best way to embark on my weight loss journey?",
        "output": "ADVICE",
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_examples = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)


(b) Setting up the intent prompt

In [ ]:
intent_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an intent classifier for the field of interest in the query.
Given a question, classify it into one of the following intents:
- DEFINE: Asking for a definition or description
- EXPLAIN: Asking for reasoning or why something is the case
- PROCEDURE: Asking for how-to or steps
- ADVICE: Asking for personalized or practical suggestions
- COMPARISON: Asking to compare options
- GENERAL: Anything else
Return only the intent, nothing else.
Here are a few examples:""",
        ),
        # few shot examples
        few_shot_examples,
        # New question
        ("user", "{question}"),
    ]
)

(c) Set up routing

In [ ]:
router = HuggingFaceChatRunnable(client, intent_prompt, 0.0, 10)

# Section 7: Stepback Generation

(a) Step-back prompt

In [ ]:
# This are examples that shows the LLM what it is achieving through stepback

examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]

# Now translate this into an example_prompt
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

step_back_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot,
        # New question
        ("user", "Intent: {intent}\nQuestion: {question}"),
    ]
)

(c) Set up Step Back

In [ ]:
stepback = HuggingFaceChatRunnable(client, step_back_prompt, 0.0, 1024)

# Section 8: Routing Chains

(a) Chains

In [ ]:
#define prompt
defineprompt = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.
You are responding to a query with the intent: DEFINE.
Your answer should be:
- Comprehensive, but concise (1–3 sentences max)
- Factually correct and aligned with the provided context
- Free of speculation, advice, or subjective judgment
- Focused only on essential information—no unnecessary background or examples unless they resolve ambiguity
- Adjusted for multiple meanings if applicable
- Written in terminology appropriate to the user's domain or field


# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""

#explain prompt
explainprompt = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.
You are responding to a query with the intent: EXPLAIN.
Your answer should be:
- Clear and logically structured
- Focused on cause, reasoning, background, or significance
- Factually correct and aligned with the provided context
- Neutral in tone—avoid persuasion, speculation, or personal opinions
- Examples are welcome from the context provided, if it helps to improve understanding.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""

#procedure prompt
procedureprompt = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.
You are responding to a query with the intent: PROCEDURE.
Your answer should be:
- Structured as a clear, ordered list of steps (e.g., 1, 2, 3...)
- Focused on how-to instructions or best-practice sequences
- Specific, practical, and applicable to the user’s likely context
- Factually accurate and based on reliable knowledge
- Aligned with the provided context; ignore context if irrelevant

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:
1.
2.
3."""

#advice prompt
adviceprompt = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.
You are responding to a query with the intent: ADVICE.
Your answer should be:
- Actionable and practical, tailored to a general user (not personalized)
- Fact-based, but sensitive to nuance, caution, or best practices
- Free from subjective judgment or emotional language
- Respectful of varying conditions or assumptions
- Aligned with the provided context; if not relevant, ignore the context

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""

#comparison
comparisonprompt = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.
You are responding to a query with the intent: COMPARISON.
Your answer should be:
- A neutral, side-by-side analysis of options or alternatives
- Factually grounded—avoid personal recommendations unless one option is clearly superior based on evidence
- Clearly structured with bullet points or short paragraphs
- Helpful in illustrating pros and cons, similarities, and differences
- Consistent with the context provided; ignore it if irrelevant

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:
Option A:
Option B: """

#general prompt
generalprompt = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.
You are responding to a query with the intent: GENERAL.
Your answer should be:
- Informative and contextually aware
- Concise but flexible in length (aim for clarity)
- Objective and based on verifiable information
- Avoid speculation or personal opinion
- Aligned with the provided context if relevant

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""


(b) Load Prompt into a Global Variable

In [ ]:
from langchain_core.runnables import RunnableLambda

intent_router = RunnableLambda(lambda x: {
    "DEFINE": HuggingFaceChatRunnable(client, defineprompt, 0.0, 1024),
    "EXPLAIN": HuggingFaceChatRunnable(client, explainprompt, 0.0, 1024),
    "PROCEDURE": HuggingFaceChatRunnable(client, procedureprompt, 0.0, 1024),
    "ADVICE": HuggingFaceChatRunnable(client, adviceprompt, 0.0, 1024),
    "COMPARISON": HuggingFaceChatRunnable(client, comparisonprompt, 0.0, 1024),
    "GENERAL": HuggingFaceChatRunnable(client, generalprompt, 0.0, 1024),
}[x["intent"].strip()]
)

(c) Chain

In [ ]:
from langchain_core.runnables import RunnableMap, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

chain = (
    RunnableMap({
        "question": lambda x: x["question"],
        "step_back_question": lambda x: x["question"],
        "intent": lambda x: router.invoke({"question": x["question"]})
    })
    | RunnableLambda(lambda x: {
        "normal_context": retriever.invoke(x["question"]),
        "step_back_q": stepback.invoke({"intent" : x["intent"],"question": x["step_back_question"]}),
        "question": x["question"],
        "intent": next(iter(x["intent"])) if isinstance(x["intent"], set) else x["intent"]
    })
    | RunnableLambda(lambda x: {
        "step_back_context": retriever.invoke(x["step_back_q"]),
        "normal_context": x["normal_context"],
        "question": x["question"],
        "intent": x["intent"]
    })
    | intent_router
    | StrOutputParser()
)

# Section 9: Running the Query

In [ ]:
question = input("Ask me anything! \n")

# Generate the Response
response = chain.invoke({"question": question})
print(response)